<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/master/src/notebooks/ResNet_Tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wide ResNet 16_4

In [0]:
import tensorflow as tf

In [0]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import  Input, Add, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import  l2
from tensorflow.keras import backend as K
import warnings

warnings.filterwarnings("ignore")
weight_decay = 0.0005

In [0]:
def initial_conv(input):
  x = Convolution2D(16,(3,3),padding="same", kernel_initializer='he_normal',
                    kernel_regularizer=l2(weight_decay),use_bias=False)(input)
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  return x

In [0]:
def expand_conv(init, base, k, strides = (1,1)):
  x = Convolution2D(base * k, kernel_size=(3,3),padding= "same", strides=strides, kernel_initializer="he_normal", kernel_regularizer=l2(weight_decay),
                    use_bias=False)(init)
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon= 1e-5, gamma_initializer= 'uniform')(x)
  x = Activation('relu')(x)

  x = Convolution2D(base * k, kernel_size=(3,3), padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(weight_decay),
                    use_bias = False)(x)
  skip = Convolution2D(base * k, kernel_size=(1,1), padding='same', strides=strides, kernel_initializer='he_normal',
                       kernel_regularizer=l2(weight_decay),
                       use_bias = False)(init)
  m = Add()([x, skip])
  return m

In [0]:
def conv1_block(input, k=1, dropout = 0.0):
  init = input
  
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
  x = Activation('relu')(x)
  x = Convolution2D(16 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay),
                    use_bias=False)(x)
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(16 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay),
                    use_bias = False)(x)
  m = Add()([init, x])
  return m

In [0]:
def  conv2_block(input, k=1, dropout = 0.0):
  init = input

  channel_axis = 1 

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
  x = Activation('relu')(x)
  x = Convolution2D(32 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer = l2(weight_decay), use_bias = False)(x)
  
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(32*k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer= l2(weight_decay),
                    use_bias = False)(x)
  
  m = Add()([init, x])
  return m


In [0]:
def conv3_block(input, k=1, dropout=0.0):
  init = input

  channel_axis = 1 

  x = BatchNormalization(axis=channel_axis, momentum= 0.1, epsilon = 1e-5, gamma_initializer='uniform' )(input)
  x = Activation('relu')(x)
  x = Convolution2D(64 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer= l2(weight_decay),
                    use_bias = False)(x)
  
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(64 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=l2(weight_decay),
                    use_bias=False)(x)
  m = Add()([init, x])
  return m


In [0]:
def create_wide_residual_network(input_dim, nb_classes=4, N=2, k=2, dropout = 0.0, verbose=1):
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  ip = Input(shape=input_dim)

  x = initial_conv(ip)
  nb_conv = 4
  x = expand_conv(x, 16, k)
  nb_conv +=2

  for i in range(N-1):
    x = conv1_block(x, k, dropout)
    nb_conv +=2
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)

  x = expand_conv(x, 32, k, strides=(1, 1))
  nb_conv += 2
  for i in range(N - 1):
    x = conv2_block(x, k, dropout)
    nb_conv += 2
  
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  
  x = expand_conv(x, 64, k, strides=(1, 1))
  nb_conv += 2
  
  for i in range(N - 1):
    x = conv3_block(x, k, dropout)
    nb_conv += 2
    
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  
  x = AveragePooling2D((8, 8))(x)
  x = Flatten()(x)
  
  x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)
  
  model = Model(ip, x)
  
  if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
  
  return model





In [110]:
if __name__ == "__main__":
    from tensorflow.keras.layers import Input
    from tensorflow.keras.models import Model

    init = (68, 100,1)
    wrn16_2 = create_wide_residual_network(init, nb_classes=4, N=2, k=2, dropout = 0.4, verbose=1)
  

Wide Residual Network-16-2 created.


In [0]:
from sklearn.model_selection import train_test_split
import gzip
import pickle
import numpy as np

In [0]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()


In [113]:
new_data_X = []
Y_data = []
for row in data:
    new_data_X.append(row['crop'])
    Y_data.append(row['label'])
new_data_X = np.array(new_data_X)
new_data_X.shape

(5722, 68, 100)

In [114]:
X = new_data_X.astype('float32')
X[0].shape

(68, 100)

In [0]:
img_rows, img_cols = X[0].shape

# transform data set
if K.image_data_format() == 'channels_first':
    X = X.reshape(X.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X = X.reshape(X.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [116]:
X.shape

(5722, 68, 100, 1)

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y_df = pd.DataFrame(Y_data, columns=['Label'])
y_df['Encoded'] = labelencoder.fit_transform(y_df['Label'])

In [118]:
y_df['Label'].value_counts()

open             1500
closed           1500
partiallyOpen    1376
notVisible       1346
Name: Label, dtype: int64

In [119]:
y_df['Encoded'].value_counts()


2    1500
0    1500
3    1376
1    1346
Name: Encoded, dtype: int64

In [0]:
from tensorflow.keras.utils import to_categorical

y_cat = to_categorical(y_df['Encoded'])

In [0]:
from tensorflow.keras.optimizers import SGD

EPOCHS = 100
BS = 128

In [0]:
sgd = SGD(lr=0.01, momentum=0.9)

In [127]:
wrn16_2.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
print("Finished compiling")

Finished compiling


In [0]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping


In [0]:
def lr_sch(epoch):
    if epoch < 60:
        return 0.01
    elif epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_sch)

In [128]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=10, random_state=42, shuffle=False)
hist_list = []
train_set= []
test_set = []

for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y_cat[train_index], y_cat[test_index]
    hist = wrn16_2.fit(X_train, y_train, 
                        batch_size=BS, epochs=EPOCHS, verbose=2, validation_data = (X_test, y_test),
                         shuffle=False, callbacks=[lr_scheduler])
    hist_list.append(hist)
    

Train Index:  [ 573  574  575 ... 5719 5720 5721] 

Test Index:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 23

KeyboardInterrupt: ignored

In [31]:

 hist = wrn28_10.fit(X_train, y_train_cat, 
                        batch_size=BS, epochs=EPOCHS, verbose=2,
                        validation_split = 0.25,
                         shuffle=False, callbacks=[lr_scheduler])

Epoch 1/160
31/31 - 16s - loss: 2.5195 - acc: 0.2730 - val_loss: 2.5052 - val_acc: 0.2616 - lr: 0.0100
Epoch 2/160
31/31 - 16s - loss: 2.4597 - acc: 0.3471 - val_loss: 2.3668 - val_acc: 0.4200 - lr: 0.0100
Epoch 3/160
31/31 - 16s - loss: 2.3458 - acc: 0.4307 - val_loss: 2.3226 - val_acc: 0.4402 - lr: 0.0100
Epoch 4/160
31/31 - 16s - loss: 2.3113 - acc: 0.4538 - val_loss: 2.3012 - val_acc: 0.4573 - lr: 0.0100
Epoch 5/160
31/31 - 16s - loss: 2.2849 - acc: 0.4626 - val_loss: 2.2671 - val_acc: 0.4643 - lr: 0.0100
Epoch 6/160
31/31 - 16s - loss: 2.2547 - acc: 0.4864 - val_loss: 2.2415 - val_acc: 0.4829 - lr: 0.0100
Epoch 7/160
31/31 - 16s - loss: 2.2230 - acc: 0.5115 - val_loss: 2.2057 - val_acc: 0.5054 - lr: 0.0100
Epoch 8/160
31/31 - 16s - loss: 2.1892 - acc: 0.5346 - val_loss: 2.1792 - val_acc: 0.5225 - lr: 0.0100
Epoch 9/160
31/31 - 16s - loss: 2.1536 - acc: 0.5540 - val_loss: 2.1694 - val_acc: 0.5078 - lr: 0.0100
Epoch 10/160
31/31 - 16s - loss: 2.1159 - acc: 0.5677 - val_loss: 2.1218 

KeyboardInterrupt: ignored

In [0]:
""" New model"""

EPOCHS_120 = 120
wrn16_4 = create_wide_residual_network(init, nb_classes=4, N=2, k=2, dropout = 0.0, verbose=1)
  
hist2 = wrn16_4.fit(X_train, y_train_cat, 
                        batch_size=BS, epochs=EPOCHS, verbose=1,
                        validation_data=(X_test,y_test_cat), shuffle=True, callbacks=[lr_scheduler])

In [0]:
#hist = wrn28_10.fit(X_train, to_categorical(y_train_df['New']), validation_split=0.33, epochs=EPOCHS, batch_size=BS, verbose=2)

In [0]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
horizontal_flip=True, fill_mode="nearest")

In [0]:
hist_1 = wrn28_10.fit_generator(aug.flow(X_train, y_train, batch_size=BS),validation_data=(X_test, y_test),callbacks =[lr_scheduler],verbose=1, epochs=EPOCHS)

In [0]:
wrn28_10.save("model_last.h5")

In [0]:
img = X_train[8]
img.shape

In [0]:
y_train[8]

In [0]:
np.argmax(wrn28_10(image))

In [0]:
y_train_df['Label'])

In [0]:
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (68, 100))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector

In [0]:
import tensorflow as tf
image = tf.convert_to_tensor(img)
image = preprocess(image)
image.shape

In [0]:
probs = wrn28_10.predict(image,batch_size=None,steps=1)

label = tf.one_hot(2, probs.shape[-1])
label = tf.reshape(label, (1, probs.shape[-1]))
label

In [0]:
probs

In [0]:
wrn28_10(image)

In [0]:
np.argmax(probs)

In [0]:
import matplotlib.pyplot as plt

history = hist_1
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
model = wrn28_10

In [0]:
!pip install -q tensorflow==2.0.0b1
# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

import cleverhans
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("\nTensorflow Version: " + tf.__version__)
print("Cleverhans Version: " + cleverhans.__version__)
print("GPU Available: ", tf.test.is_gpu_available())

In [0]:
# Import the attack
from cleverhans.future.tf2.attacks import fast_gradient_method

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input,model.layers[-1].output)

In [0]:
epsilon = 0.05

adv_example_untargeted_label = fast_gradient_method(logits_model, image, epsilon, np.inf, targeted=False)

adv_example_untargeted_label_pred = model.predict(adv_example_untargeted_label)

In [0]:
np.argmax(adv_example_untargeted_label_pred)